In [142]:
#imports
from tkinter import *
import os
import re
import time
import datetime

import telnetlib
import getpass 
import sys
import time
import serial
import threading #potentially replaced by multiprocessing
from multiprocessing.pool import ThreadPool

import pandas as pd
import numpy as np

from pdfrw import PdfReader
import matplotlib.pyplot as plt
import PyPDF2
from tkinter.filedialog import askopenfilename
os.environ.setdefault('MAGICK_HOME', '/usr/local/Cellar/imagemagick@6/6.9.9-34/')
from wand.image import Image
from PIL import Image as PI
import pyocr
import pyocr.builders
import io


#global defines
nport_ip = "10.136.1.99"
ref_port = "4001"

#global variables
rma_port = "4002"
ins_type = "sbe3703"
ref_type = "sbe19"

#for test purposes
diag_file = open("diagnostic_values.txt", "w")


In [3]:

#updates the main status label with given text.  Could be accomplished by just calling "main_status.config" from within the main function...
def update_txt(next_instruction):
    main_status.config(text=next_instruction)   
    
#def connex(port):
    #print(port)
    
win = Tk()

win.title('SBE CTD Prequalification')
win.geometry('1000x600') # Size 200, 200

text = Text(win, height=2)

#labels
main_status = Label(win, text="Waiting for device selection")
calsheet_status_label = Label(win, text="")
sample_status_label = Label(win, text="")

#Buttons
port2 = Button(win, text="4002 - 485", command=lambda: instrument_connection("4002"))
port3 = Button(win, text="4003 - 232", command=lambda: instrument_connection("4003"))
port4 = Button(win, text="4004", command=lambda: instrument_connection("4004"))
#exit_button = Button(win, text="Exit", command=win.destroy())

text.insert(INSERT, "Please connect the RMA device and specify the port used")

text.pack()
port2.pack()
port3.pack()
port4.pack()
main_status.pack()
calsheet_status_label.pack()
sample_status_label.pack()

mainloop()

03TA0=None

03TA1=None

03TA2=1234.23

03TA3=None

03G=-1.017709e+000

03H=1.328397e-001

03I=-1.413619e-004

03J=None

03CPCOR=-9.5700e-008

03CTCOR=3.2500e-006

03WBOTC=None

03reference pressure=None



In [1]:
# Instrument Connection
#module to:
# 1. Prompt user to select NPORT port RMA instrument is connected to (maybe goes in main gui window?)
# 2. Create a telnet connection to both instruments
# 3. Send wakeup to both instruments
# 4. Alert user that connection successful



#entry point to module
def instrument_connection(rmar_port):
    global nport_ip
    global ref_port
    global rma_port
    global ins_type
    rma_port = rmar_port
    ins_type = get_ins_type()
    device_dict = {
        "4002" : "id?\r\n",
        "4003" : "\r\n"
    }
    try:
        ref_telnet = telnetlib.Telnet(nport_ip, ref_port) #open connection
        rma_telnet = telnetlib.Telnet(nport_ip, rma_port)
        update_txt("Connecting, please wait...")
        time.sleep(1)
        ref_telnet.write(device_dict['4003'].encode("ascii"))  #wakeup both instruments
        rma_telnet.write((device_dict[rma_port]).encode("ascii"))
        time.sleep(1)
        rma_telnet.write((device_dict[rma_port]).encode("ascii"))
        time.sleep(1)
        rma_ok = rma_telnet.read_very_eager()
        ref_ok = ref_telnet.read_very_eager()
        ref_telnet.close() #close reference connection, not done with rma yet
        rma_telnet.close()
    except:
        print("bricked it")
        ref_telnet.close()
        rma_telnet.close()
    finally:
        ref_telnet.close()
        rma_telnet.close()
    update_txt("Connection Successful")
    calsheet_status_label.config(text="Collecting RMA calibration data...")    
    compcals = compare_cals_entrypoint(rma_port) #pass the telnet connection to compare_cals function
    gen_report(compcals)    
    
#get ins type
#sends an ID query, if 37 ->returns "sbe37##" where ## is the id, if 19 no response should come from the instrument so
#we can default to "sbe19"
    
def get_ins_type():
    global nport_ip
    global ref_port
    global rma_port
    global ins_type
    rma_telnet = telnetlib.Telnet(nport_ip, rma_port)
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    ins_type = rma_telnet.read_very_eager()
    ins_type = ins_type.decode("unicode-escape")
    if ins_type[:2] == "id":
        idd = "ins_type = sbe37" + ins_type[3:5]
        diag_file.write(idd)
        return "sbe37" + ins_type[3:5]
    else:
        diag_file.write("ins_type = sbe19")
        return "sbe19"
    
    
# Compare Cals
#module to:
# 1. Prompt user for calibration sheet location
# 2. Collect cals from instrument(dcal, #iidc)
# 3. Compare Results
# 4. Display side by side results AND suggest an action(pass/fail)
# 5. If "pass" then continue after ~30seconds if "fail" pause and wait for user?\
def conv_to_float(x):  #lambda float conversion
    try:
        return float(x)
    except:
        return 0

def compare_cals_entrypoint(rma_port):
    instr_test = 'SBE37SI-RS485 V 3.2  5679\r\ntemperature:  25-Mar-17\r\n    TA0 = 4.261456e-05\r\n    TA1 = 2.731360e-04\r\n    TA2 = -2.206636e-06\r\n    TA3 = 1.490857e-07\r\nconductivity:  25-Mar-17\r\n    G = -9.994807e-01\r\n    H = 1.566288e-01\r\n    I = -3.317297e-04\r\n    J = 4.905268e-05\r\n    CPCOR = -9.570000e-08\r\n    CTCOR = 3.250000e-06\r\n    WBOTC = 1.185617e-06\r\nreference pressure = 2007.0 decibars\r\n<Executed/>\r\n'
    sheet_test = open("Output.txt", 'r').read()
    if rma_port == "4002":
        ins_caldata = get_cals_37()
    if rma_port == "4003":
        ins_caldata = get_cals_19()
    sheet_caldata = get_sheet_cals()
    cal_values = compare_cals(ins_caldata, sheet_caldata)
    #update_cals(df)  ##THOROUGH TEST BEFORE FIRING THIS ONE
    return cal_values
    
def get_cals_37():
    global nport_ip
    rma_telnet = telnetlib.Telnet(nport_ip, "4002")
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    sbe_id = str(rma_telnet.read_very_eager())[5:7]
    rma_telnet.write(("#"+sbe_id+"DC\r\n").encode("ascii"))
    time.sleep(1)
    caldata = rma_telnet.read_very_eager()
    rma_telnet.close()
    return caldata #.decode('unicode-escape')
def get_cals_19():
    rma_telnet = telnetlib.Telnet(nport_ip, "4002")
    rma_telnet.write("DCal\r\n".encode("ascii"))
    time.sleep(1)
    rma_telnet.write("DCal\r\n".encode("ascii"))
    time.sleep(1)
    caldata = rma_telnet.read_very_eager()
    rma_telnet.close()
    return str(caldata)

#this function doesn't actually need to exist, but helps with clarity i think?
def get_sheet_cals():
    filepath = request_filepath()
    return read_file(filepath)
    

#get user input filepath(browse files)    
def request_filepath():
    file_select = Tk()
    file_select.withdraw()
    filename=askopenfilename()
    file_select.update()
    file_select.destroy()
    return filename

#OCR on file, parse out to text
def read_file(filepath):
    tool = pyocr.get_available_tools()[0]
    lang = tool.get_available_languages()[0]
    req_image = []
    final_text = []
    image_pdf = Image(filename=filepath, resolution=300)
    image_jpeg = image_pdf.convert('jpeg')
    for img in image_jpeg.sequence:
        img_page = Image(image=img)
        req_image.append(img_page.make_blob('jpeg'))
    for img in req_image: 
        txt = tool.image_to_string(
            PI.open(io.BytesIO(img)),
            lang=lang,
            builder=pyocr.builders.TextBuilder()
        )
        final_text.append(txt)
    merge_text = ''.join(final_text)
    return merge_text

def split_sheet(sheet):
    try:
        m = sheet[0].split(' ')[0]
        return m
    except:
        pass

def compare_cals(instr, sheet):
    i_s = instr
    s_s = sheet
    comparison_array = []
    instrument = parse_cal_tags(i_s.decode('unicode-escape'))
    sheet_return = parse_sheets(s_s, instrument)     
    sheet_return = clean_sheet_data(sheet_return)
    return (instrument, sheet_return)
    
def find_value(tag, input_file):
    m = re.findall(tag, input_file)
    if m:
        return m
    else:
        return "0"

def split_sheet(sheet):
    try:
        m = sheet[0].split(' ')[0]
        return m
    except:
        pass

def parse_sheets(s_s, instr):    
    sheet_return = []
    instrument = instr
    for i in range(len(instrument[0])):
        tags = instrument[0]
        sheet_regex = r"(?i)(?<=\s" + instrument[0][i] + " = )(.*)(?=\s)"
        m = re.findall(sheet_regex, s_s)
        sheet_return.append(split_sheet(m))
    return sheet_return
    
                   
def parse_cal_tags(cal_string):
    m = re.findall(r"(?<=\s)(.*)(?=\s)", cal_string) 
    calibration_tags = []
    calibration_rows = []
    for x in m:
        entry = x.split('=')
        try:
            if len(entry) >= 2:
                calibration_tags.append(entry[0].strip())
                calibration_rows.append(entry[1].strip())
        except:
            pass
    return (calibration_tags, calibration_rows)

def clean_sheet_data(sheet_data):
    cleanup_dict = {"~":"-",
                   "€":"e",
                   "|":"1",
                   "—":"-",
                   "l":"1",
                   "O":"0",
                   "0":"0"
                   }
    cleanup_list = ["~",                    
                    "€",
                   "|",
                   "—",
                   "l",
                   "O",
                   "0"]
    cleaned_list=[]
    for i in sheet_data:
        clean = i
        if clean:
            for i in range(len(cleanup_list)):
                clean = clean.replace(cleanup_list[i], cleanup_dict[cleanup_list[i]])
        cleaned_list.append(clean)
    return cleaned_list

##Helper function to clear whatever is on the frame currently  
def clear_screen():
    for widget in win.winfo_children():
        widget.destroy()

#### Generates the Calibration Report with option to update values
###
##
#
def gen_report(cals):
    def stringvar_list(lst):
        vals = [str(i) for i in lst]
        ret_lst = [None]*len(lst)
        for i in range(len(lst)):
            ret_lst[i] = StringVar()
            ret_lst[i].set(vals[i])
        return ret_lst
    def update_cal_var():
        for i in range(length):
            try:
                sheet_values[i].set(update_val[i].get())
            except Exception as e:
                print (e)
    def update_instrument_cals(not_needed):
        global rma_port
        global ins_type
        keys = [None]*len(instr_tags)
        values = [None]*len(instr_tags)
        for i in range(len(instr_tags)):
            keys[i] = instr_tags[i].get()
            values[i] = sheet_values[i].get()
        instr_data_dict = dict(zip(keys, values))
        if ins_type[:5] == "sbe37":
            for key, value in instr_data_dict.items():      
                if value != "None":
                    writestr = "#" + ins_type[5:] + key + "=" + value + "\r\n" #we see the prepension again
                    print(writestr)  #this print statement is for test purposes, commented out section should work(untested)
    #                 rma_telnet = telnetlib.Telnet(nport_ip, rma_port)
    #                 rma_telnet.write(writestr.encode("ascii"))
    #                 time.sleep(1)
        else:
            for key, value in instr_data_dict.items():            
                if value != "None":
                    writestr = key + "=" + value + "\r\n"
                    print(writestr)
    #                 rma_telnet = telnetlib.Telnet(nport_ip, rma_port)
    #                 rma_telnet.write(writestr.encode("ascii"))
    #                 time.sleep(1)
       rma_telnet.close()
       ref_telnet.close()
        
    clear_screen()
                
    instr_data = cals[0]
    sheet_data = cals[1]
       
    sheet_data = ['None' if v is None else v for v in sheet_data]

    instr_tags = stringvar_list(instr_data[0])
    instr_values = stringvar_list(instr_data[1])
    sheet_values = stringvar_list(sheet_data)
    
    length = len(instr_data[0])
    tags = [None] * length
    ins_val = [None] * length
    she_val = [None] * length
    update_entry = [None] * length
    update_val = [None]*length
    
    cal_label = Label(win, text="Calibration Comparison", height=1, width=20)
    cal_label.grid(sticky='NW', columnspan=3)
          
    for i in range(length):
        tags[i] = Text(win, height=1, width=15)
        tags[i].insert(INSERT, instr_tags[i].get())
        tags[i].grid(column=0, row=i+1)
        ins_val[i] = Text(win, height=1, width=15)
        ins_val[i].insert(INSERT, instr_values[i].get())
        ins_val[i].grid(column=1, row=i+1, sticky='NW')
        she_val[i] = Label(win, height=1, width=15, text=sheet_values[i].get() , textvariable=sheet_values[i])
        she_val[i].grid(column=2, row=i+1, sticky='NW', padx = 10)
        update_val[i] = StringVar()
        update_entry[i] = Entry(win ,textvariable = update_val[i])
        update_entry[i].insert(END, sheet_values[i].get())
        update_entry[i].grid(column=3, row=i+1, sticky='NW')
    
    update_button = Button(win, text='Update Values', command=update_cal_var)
    update_button.grid(row=length+1, columnspan=3)
    
    #update_ins_text
    
    update_instrument = Button(win, text="Update Instruments", command=lambda: update_instrument_cals("sbe3703"))
    update_instrument.grid(row=length+3, columnspan=3)
        

In [6]:
s = "sbe3703"
s[5:]

'03'

In [7]:
s = ["1","2","3","4"]
d = ["a","b","c","d"]
dicc = dict(zip(s,d))
for key, value in dicc.items():
    print(key + " "+ value +"\r\n")

1 a

2 b

3 c

4 d



In [138]:
#diag_file.close()
rma_telnet.close()
#ref_telnet.close()

In [127]:
#sample from 19+
caldata = []
logging_list = ["STOP","STOP","ECHO=y","MP","MP","OUTPUTFORMAT=3","IGNORESWITCH=Y","AUTORUN=Y","OUTPUTSAL=Y","OUTPUTSV=Y","OUTPUTUCSD=N","NAVG=4","MINCONDFREQ=500","PUMPDELAY=60","INITLOGGING","INITLOGGING","STARTNOW"]
rma_telnet = telnetlib.Telnet(nport_ip, "4001")
for x in logging_list:
    wr_st = x + "\r\n"
    rma_telnet.write(wr_st.encode("ascii"))
    time.sleep(1)
time.sleep(10)
#caldata = rma_telnet.read_very_eager()
for i in range(30):
    sample_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S') # :%f')
    caldata.append(str(rma_telnet.read_until(b"\r\n"))+ ", " + sample_time)
    time.sleep(4)
    
rma_telnet.close()

samples = ''.join(caldata).split("<Executed/>")[-1:][0].split("b")
samples

[" 21.7521,  0.00220,   -0.037, 17.282, 0.604525,   0.0181, 1487.632\\r\\n' -==- 2018/04/05 10:07:27:977474",
 "' 21.7522,  0.00220,   -0.038, 17.282, 0.604525,   0.0181, 1487.632\\r\\n' -==- 2018/04/05 10:07:31:979159",
 "' 21.7525,  0.00220,   -0.037, 17.282, 0.604525,   0.0181, 1487.633\\r\\n' -==- 2018/04/05 10:07:35:980884",
 "' 21.7525,  0.00220,   -0.037, 17.284, 0.604521,   0.0181, 1487.633\\r\\n' -==- 2018/04/05 10:07:39:982524",
 "' 21.7526,  0.00220,   -0.038, 17.286, 0.604516,   0.0181, 1487.634\\r\\n' -==- 2018/04/05 10:07:43:984283",
 "' 21.7526,  0.00220,   -0.037, 17.286, 0.604516,   0.0181, 1487.634\\r\\n' -==- 2018/04/05 10:07:47:984835",
 "' 21.7529,  0.00220,   -0.038, 17.286, 0.604516,   0.0181, 1487.635\\r\\n' -==- 2018/04/05 10:07:51:985852",
 "' 21.7530,  0.00220,   -0.038, 17.286, 0.604511,   0.0181, 1487.635\\r\\n' -==- 2018/04/05 10:07:55:986664",
 "' 21.7531,  0.00220,   -0.038, 17.285, 0.604495,   0.0181, 1487.635\\r\\n' -==- 2018/04/05 10:07:59:988373",
 "

In [139]:
#sample from 37
rma_telnet= telnetlib.Telnet(nport_ip, "4002") 
sample_pack = []
for i in range(30):
    rma_telnet.write("#03TS\r\n".encode("ascii"))
    time.sleep(4)
    sample_pack.append(rma_telnet.read_very_eager())
rma_telnet.close()
sample_pack

[b'03, 03705679,  21.6589, -0.00001,1986.440, 1520.781,  6.8024, 05 Apr 2018, 18:19:31\r\n<Executed/>\r\n',
 b'03, 03705679,  21.6592, -0.00002,1986.440, 1520.782,  6.8022, 05 Apr 2018, 18:19:35\r\n<Executed/>\r\n',
 b'03, 03705679,  21.6595, -0.00002,1986.440, 1520.783,  6.8022, 05 Apr 2018, 18:19:39\r\n<Executed/>\r\n',
 b'03, 03705679,  21.6595, -0.00002,1986.440, 1520.783,  6.8021, 05 Apr 2018, 18:19:43\r\n<Executed/>\r\n',
 b'03, 03705679,  21.6595, -0.00002,1986.440, 1520.783,  6.8022, 05 Apr 2018, 18:19:47\r\n<Executed/>\r\n',
 b'03, 03705679,  21.6600, -0.00002,1986.440, 1520.784,  6.8021, 05 Apr 2018, 18:19:51\r\n<Executed/>\r\n',
 b'03, 03705679,  21.6594, -0.00002,1986.440, 1520.782,  6.8022, 05 Apr 2018, 18:19:55\r\n<Executed/>\r\n',
 b'03, 03705679,  21.6602, -0.00002,1986.440, 1520.785,  6.8021, 05 Apr 2018, 18:19:59\r\n<Executed/>\r\n',
 b'03, 03705679,  21.6599, -0.00002,1986.440, 1520.784,  6.8021, 05 Apr 2018, 18:20:03\r\n<Executed/>\r\n',
 b'03, 03705679,  21.6607, -

In [152]:
###highly speculative UPDATE INSTRUMENT CALS section
#ins_type should contain a string either sbe37#ii where the first # is a literal # sign and ii is the id number, or sbe19 with no number
#cal data will be a dictionary with key value pairs of tag:value
#port parameter will specify which port the instrument is connected to.
#def update_instrument_cals(ins_type, cal_data, port):

##is now incorporated - ready for CAREFUL testing CAREFULLLL
rma_telnet = telnetlib.Telnet(nport_ip, "4001")
rma_telnet.write("STOP\r\n".encode("ascii"))
rma_telnet.close()

In [192]:
### Sampling checks
### to be done after the cals are set properly 
# take 10 samples each instrument
# compare ---> what does this involve
# generate a report -> graph with proper scaling/table/average/outlier etc?  mostly a nice to have 

def get_samples():
    global nport_ip
    global ref_port
    global rma_port
    global ins_type
    
    
    #begin threading for sample simultaneously
    pool = ThreadPool(processes=2)
    procRMA = pool.apply_async(sample_from_ins, args=(ins_type,rma_port)) 
    procREF = pool.apply_async(sample_from_ins, args=(ref_type,ref_port))
    rma_sample = procRMA.get()
    ref_sample = procREF.get()
    print(rma_sample)
    print("================================================")
    print(ref_sample)
    return (rma_sample, ref_sample)
#    return sample_todf(ref_sample, rma_sample, rma_port)
    
    #collect data for 10 samples
    
def sample_from_ins(itype, port):
    if itype[:5] == "sbe19":
        caldata = []
        logging_list = ["STOP","STOP","ECHO=y","MP","MP","OUTPUTFORMAT=3","IGNORESWITCH=Y","AUTORUN=Y","OUTPUTSAL=Y","OUTPUTSV=Y","OUTPUTUCSD=N","NAVG=4","MINCONDFREQ=500","PUMPDELAY=60","INITLOGGING","INITLOGGING","STARTNOW"]
        rma_telnet = telnetlib.Telnet(nport_ip, port)
        for x in logging_list:
            wr_st = x + "\r\n"
            rma_telnet.write(wr_st.encode("ascii"))
            time.sleep(1)
        time.sleep(5)
        #caldata = rma_telnet.read_very_eager()
        for i in range(33):
            sample_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S') # :%f')
            caldata.append(str(rma_telnet.read_until(b"\r\n"))+ ", " + sample_time)
            time.sleep(4)
        rma_telnet.write("STOP\r\n".encode("ascii"))
        rma_telnet.close()
        samples = ''.join(caldata).split("<Executed/>")[-1:][0].split("b")
    else:
        time.sleep(98)
        rma_telnet= telnetlib.Telnet(nport_ip, port) 
        samples = []
        for i in range(30):
            rma_telnet.write(("#"+ ins_type[5:] + "TS\r\n").encode("ascii"))
            time.sleep(4)
            sample_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S') # :%f')
            samples.append(str(rma_telnet.read_very_eager())+sample_time)
        rma_telnet.close()
    return samples
        
    
samples = get_samples()
#sample_from_ins(ins_type, rma_port)

["b''2018/04/05 15:30:50", "b'03, 03705679,  22.0004, -0.00001,1986.440, 1521.801,  6.7159, 05 Apr 2018, 22:36:57\\r\\n<Executed/>\\r\\n'2018/04/05 15:30:54", "b'03, 03705679,  21.9997, -0.00002,1986.440, 1521.799,  6.7161, 05 Apr 2018, 22:37:01\\r\\n<Executed/>\\r\\n'2018/04/05 15:30:58", "b'03, 03705679,  21.9978, -0.00002,1986.440, 1521.794,  6.7165, 05 Apr 2018, 22:37:05\\r\\n<Executed/>\\r\\n'2018/04/05 15:31:02", "b'03, 03705679,  21.9968, -0.00002,1986.440, 1521.791,  6.7167, 05 Apr 2018, 22:37:09\\r\\n<Executed/>\\r\\n'2018/04/05 15:31:06", "b'03, 03705679,  21.9983, -0.00002,1986.440, 1521.795,  6.7164, 05 Apr 2018, 22:37:13\\r\\n<Executed/>\\r\\n'2018/04/05 15:31:10", "b'03, 03705679,  21.9990, -0.00002,1986.440, 1521.797,  6.7161, 05 Apr 2018, 22:37:17\\r\\n<Executed/>\\r\\n'2018/04/05 15:31:14", "b'03, 03705679,  21.9994, -0.00002,1986.440, 1521.799,  6.7161, 05 Apr 2018, 22:37:21\\r\\n<Executed/>\\r\\n'2018/04/05 15:31:18", "b'03, 03705679,  21.9993, -0.00002,1986.440, 152

In [191]:
rma_samples = samples[0][1:]
ref_samples = samples[1]
ref_samples = [ref_samples[i][1:].replace("\\r\\n'","") for i in range(len(ref_samples))]
rma_samples = [rma_samples[i][2:].replace("\\r\\n<Executed/>\\r\\n'", ", ") for i in range(len(rma_samples))]
sample_pairs = []

for i in range(len(rma_samples)):
    for j in range(len(ref_samples)):
        if rma_samples[i][-4:] == ref_samples[j][-4:]:
            sample_pairs.append((rma_samples[i], ref_samples[i]))

#sample_pairs

ref_samples

['21.8865,  0.00286,   -0.051, 17.190, 0.608342,   0.0209, 1488.028, 2018/04/05 14:27:44',
 ' 21.8867,  0.00286,   -0.053, 17.190, 0.608342,   0.0210, 1488.028, 2018/04/05 14:27:48',
 ' 21.8868,  0.00286,   -0.052, 17.193, 0.608337,   0.0210, 1488.029, 2018/04/05 14:27:52',
 ' 21.8868,  0.00287,   -0.052, 17.196, 0.608332,   0.0210, 1488.029, 2018/04/05 14:27:56',
 ' 21.8868,  0.00287,   -0.052, 17.196, 0.608332,   0.0210, 1488.029, 2018/04/05 14:28:00',
 ' 21.8870,  0.00287,   -0.052, 17.196, 0.608332,   0.0210, 1488.030, 2018/04/05 14:28:04',
 ' 21.8870,  0.00286,   -0.052, 17.197, 0.608329,   0.0210, 1488.029, 2018/04/05 14:28:08',
 ' 21.8870,  0.00287,   -0.053, 17.202, 0.608321,   0.0210, 1488.029, 2018/04/05 14:28:12',
 ' 21.8869,  0.00287,   -0.052, 17.202, 0.608321,   0.0210, 1488.029, 2018/04/05 14:28:16',
 ' 21.8871,  0.00287,   -0.052, 17.202, 0.608321,   0.0210, 1488.030, 2018/04/05 14:28:20',
 ' 21.8872,  0.00287,   -0.051, 17.202, 0.608321,   0.0210, 1488.030, 2018/04/05 

In [185]:
a = ["asdr","asde","asdf"]
b = ["Asdf","asdc","asde"]
c = []
# for i in range(len(a)):
#     for j in range(len(b)):
#         if a[i][-2:] == b[j][-2:]:
#             c.append((a[i], b[j]))
#[c.append((a[i], b[j])) if any([a[i][-2:] == b[j][-2:] for j in range(len(b))]) for i in range(len(a))]

c

SyntaxError: invalid syntax (<ipython-input-185-8494b5a820f2>, line 8)

In [180]:
c = {}
c["a"] = "b"
c

{'a': 'b'}